In [4]:
list.files("/kaggle/input/cyclism-full-data-queried/CyclismFullData.csv")

CyclismFullData <- read.csv("/kaggle/input/cyclism-full-data-queried/CyclismFullData.csv")

head(CyclismFullData)


character(0)

In [13]:
install.packages("tidyverse")
install.packages("ggplot2")
install.packages("dplyr")
library(tidyverse)
library(ggplot2)
library(dplyr)
CyclismFulldata<- cyclismFulldata.csv

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“unable to access index for repository http://cran.rstudio.com/src/contrib:
  cannot open URL 'http://cran.rstudio.com/src/contrib/PACKAGES'”
Warning message:
“package ‘tidyverse’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [12]:
#day of week mutations
CyclismFullData<-CyclismFullData %>%
mutate(
  numeric_ride_time = as.numeric(ride_time),
  day_of_week = tolower(day_of_week),
  day_of_week = factor(day_of_week,
                              levels = c("monday","tuesday","wednesday","thursday",
                                         "friday","saturday","sunday"))) 

# member casual by day of week
# Summarizing the data to count day_of_week by member_casual
CyclismSummary <- CyclismFullData %>%
  group_by(day_of_week, member_casual) %>%
  summarize(count = n(), .groups = 'drop')

# Creating the plot
CyclismDayOfWeek <- ggplot(CyclismSummary) +
  geom_line(mapping = aes(x = day_of_week, y = count, colour = member_casual, group = member_casual)) +
  labs(x = "Day of Week", y = "Count", colour = "Type")

# Printing the plot
print(CyclismDayOfWeek)


ERROR: Error in CyclismFullData %>% mutate(numeric_ride_time = as.numeric(ride_time), : could not find function "%>%"


In [ ]:
# Calculating the percentage of members and casual riders
rider_percentages <- CyclismFullData %>%
  count(member_casual) %>%
  mutate(percentage = n / sum(n) * 100)


rider_percentages_graph <- ggplot(rider_percentages, mapping =aes(x = member_casual, y = percentage, fill = member_casual))+
  geom_bar(stat = "identity")+ 
  ggtitle("Percentage of Members vs Casual Riders ")

print(rider_percentages_graph)


Another key difference being shown here, with members tending to use the service for work instead of during the weekends. Looking at times of when bikes are used during the week for members would also help provide insight into this

In [ ]:
# month_counts dataset
CyclismFullData$month <- as.numeric(CyclismFullData$month)
month_counts <- CyclismFullData %>% 
  count(member_casual,month,name = "count") %>% 
  arrange(month, member_casual)

#month_counts graph

month_counts_graph <- ggplot(month_counts) +
  geom_line(mapping = aes(x = month, y = count, group = member_casual, colour = member_casual)) +
labs(x = "month", y = "count", colour = "type")+
  ggtitle("Number of Riders By month (Jan - Mar)")+
  scale_y_continuous(labels = scales::comma)

print(month_counts_graph)


In [ ]:
#month counts percentage change dataset

# Calculate month-over-month change
month_counts_change <- month_counts %>% 
  arrange(member_casual) %>% 
arrange(month) %>% 
  mutate(
    change = count- lag(count),
    percentage_change = (count/lag(count)) * 100
  )

month_counts_graph <- ggplot(month_counts_change) +
  geom_line(mapping = aes(x = month, y = percentage_change, group = member_casual, colour = member_casual))+
  labs(x = "month", y = "percentage change", colour = "type") +
  ggtitle("Percentage Change in Riders By Month")
print(month_counts_graph)




There is a higher increase in use by casual riders as time goes past, from January to April. This is likely due to a few reasons, however my thinking is that partially this is because non members are more able to rely on other methods on transportation during Chicago's harsh winter months. Tourism and other factors also play into this - something that requires more looking into.

In [ ]:
#day of week mutations
CyclismFullData<-CyclismFullData %>% 
mutate(
  numeric_ride_time = as.numeric(ride_time),
  day_of_week = tolower(day_of_week),
  day_of_week = factor(day_of_week,
                              levels = c("monday","tuesday","wednesday","thursday",
                                         "friday","saturday","sunday"))) %>% 
arrange(day_of_week)

#ride times by day of week dataframe
ride_times<-CyclismFullData %>% 
  group_by(day_of_week, member_casual) %>% 
  summarise(avg_ride_time = mean(numeric_ride_time, na.rm = TRUE)) %>% 
  arrange(member_casual, (day_of_week))
  
  View(ride_times)
  
#ride time graphing
  
CyclismRideTime<-ggplot(ride_times) +
geom_line(mapping = aes(x = day_of_week, y = avg_ride_time, colour = member_casual, group = member_casual))+
  labs(x = "day of week", y = "average ride time", colour = "type") + 
ggtitle("Ride Time By Day of Week and Rider Type" )

print(CyclismRideTime)
 

This showed me something quite interesting, that typically members ride for shorter distances. Whilst members are more likely to be daily users who use bikes to commute, the casual users are more likely to be tourists - presumably taking more time in order to explore Chicago.

In [ ]:
#started at ended at data frame

CyclismStartEnd<- CyclismFullData %>% 
  filter(complete.cases(end_station_name)) %>% 
  group_by(start_station_name, end_station_name, member_casual) %>% 
  summarise(TripCount = n(), .groups = "drop") %>% 
  arrange(member_casual) %>% 
  group_by(start_station_name) %>% 
    mutate(
      StationPercentage = TripCount/ (sum(TripCount)*100)) %>% 
    ungroup()

view(CyclismStartEnd)
#checking for completness, that adds to 100%
  CyclismCheck<- CyclismStartEnd %>% 
group_by(start_station_name) %>% 
  summarise( TotalPercentage = sum(StationPercentage)*100)
    
view(CyclismCheck)

head(CyclismStartEnd)

I am planning on doing some work with this on a later date in tableau to get some insights to create a map into the most used locations by member_casual. I tried creating a heatmap for this but it was had too many instances and didnt really work as a result. Creating a dashboard and looking at ending stations based on a given begining station could also give some interesting results.